# Heiko's Challenge

Given and ODE that describes population level dynamics, design an ABM that has the same dynamics. The definition of same dynamics is $V(t) = P$ solve for $t$ within 20%. 

The given equation is

$$
  \frac{dV}{dt} = \lambda V \left(1-\frac{V}{k}\right)
$$

- $V_0 = 1$
- $\lambda = ln 2$
- $k = 1000$


In [1]:
using DifferentialEquations

┌ Info: Precompiling DifferentialEquations [0c46a032-eb83-5123-abaf-570d42b7fbaa]
└ @ Base loading.jl:1260


In [2]:
function solve(f, param, initial, tdomain)
    tsamples = 100
    tstart = first(tdomain)
    tend = last(tdomain)
    tlen = tend - tstart
    dt = tlen/(tsamples-1)
    tset = tstart:dt:tend
    @show tset
    sol = [initial]
    for t in 0:dt:tend
        u = sol[end] + dt*f(sol[end], t, param)
        push!(sol, u)
    end
    return sol
end

solve (generic function with 1 method)

In [3]:
V₀ = 1.0
λ = log(2)
k = 1000
f(u,t, p) = begin
    λ = first(p)
    k = last(p)
    λ*u*(1-(u/k))
end

tdomain = (0, 21)

(0, 21)

In [4]:
sol = solve(f, (λ, k), V₀, tdomain)

tset = 0.0:0.21212121212121213:21.0


101-element Array{Float64,1}:
   1.0
   1.1468841888986576
   1.3153185739908013
   1.5084570955029775
   1.7299128216190918
   1.983824009091136
   2.2749294237983895
   2.6086541414191298
   2.991207184009446
   3.4296924904646002
   3.9322348645189606
   4.508122688461221
   5.167969327283415
   ⋮
 997.8393008179573
 998.1563046219674
 998.426885612544
 998.6578186837932
 998.8548963708434
 999.0230695581017
 999.166568507365
 999.2890068275431
 999.3934706952435
 999.4825953494383
 999.5586306251345
 999.6234970601441

In [5]:
yet = false
for (i, u) in enumerate(sol)
    t = (tdomain[2]-tdomain[1])i/length(sol)
    if u > 800 && ! yet
        println("$t:\t", u)
        yet = true
    end
end

12.475247524752476:	802.4081057785716


In [7]:
using Plots

┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1260


## ABM Version

Deterministic ABM that has the same dynamics as the ODE logistic growth

In [8]:
function transition!(agents, param)
    V = length(agents)
    λ, k = first(param), last(param)
    dV = ceil(λ*V*(1-(V/k)))
    append!(agents, [:V for i in 0:dV])
    return agents
end


transition! (generic function with 1 method)

In [10]:
agents = [:V]
day = 0
while length(agents) < 999
    println(day, ":\t", length(agents))
    agents = transition!(agents, (λ, k))
    day += 1
end

0:	1
1:	3
2:	7
3:	13
4:	23
5:	40
6:	68
7:	113
8:	184
9:	290
10:	434
11:	606
12:	773
13:	896
14:	962
15:	989
16:	998


## Stochastic ABM 

This version is a true ABM, it models the agents as cells that divide instantaneously.

In [11]:
function transition_stoch!(agents, param, step)
    V = length(agents)
    λ, k = first(param), last(param)
    EdV = (λ*V*(1-(V/k)))*step
    p = EdV/V
    for i in 1:length(agents)
        add = rand(Float64) < p
        if add
            push!(agents, :V)
        end        
    end
    return agents
end

transition_stoch! (generic function with 1 method)

In [12]:
agents = [:V]
day = 0
dt = 1/2
yet = false
for i in 1:ceil(30/dt)
    println(day, ":\t", length(agents))
    agents = transition_stoch!(agents, (λ, k), dt)
    day += dt
    if length(agents) > 800 && !false
        println("============\n$day:\t", length(agents), "\n================")
        yet = true
    end
    if length(agents) >= k
        break
    end
end

0:	1
0.5:	1
1.0:	1
1.5:	1
2.0:	1
2.5:	1
3.0:	2
3.5:	2
4.0:	3
4.5:	4
5.0:	4
5.5:	5
6.0:	6
6.5:	8
7.0:	14
7.5:	17
8.0:	24
8.5:	32
9.0:	41
9.5:	60
10.0:	80
10.5:	105
11.0:	137
11.5:	176
12.0:	223
12.5:	280
13.0:	345
13.5:	421
14.0:	503
14.5:	588
15.0:	667
15.5:	746
16.0:	813
16.0:	813
16.5:	884
16.5:	884
17.0:	918
17.0:	918
17.5:	943
17.5:	943
18.0:	961
18.0:	961
18.5:	976
18.5:	976
19.0:	983
19.0:	983
19.5:	989
19.5:	989
20.0:	992
20.0:	992
20.5:	998
20.5:	998
21.0:	998
21.0:	998
21.5:	999
21.5:	999
22.0:	1001


$$\frac{dv}{dt} = λV\log\left(\frac{k}{v}\right)$$

In [13]:
f(u, t, p) = begin
    λ, k = p
    V = u
    λ*V*log(k/V)
end

f (generic function with 1 method)

In [14]:
sol = solve(f, (λ, k), V₀, tdomain)

tset = 0.0:0.21212121212121213:21.0


101-element Array{Float64,1}:
   1.0
   2.0156556869506614
   3.85513289828604
   7.005743932159319
  12.115912516413033
  19.977727216474033
  31.471979427712903
  47.476445813911894
  68.74973032659722
  95.81265655682066
 128.85280938170206
 167.67350645981645
 211.69769996668276
   ⋮
 999.9914953822414
 999.9927458212497
 999.9938124081337
 999.9947221745008
 999.9954981775757
 999.9961600845296
 999.9967246709024
 999.9972062457472
 999.9976170142698
 999.9979673871518
 999.9982662443952
 999.9985211603762

In [15]:
function transition_stoch!(agents, param, step)
    V = length(agents)
    λ, k = first(param), last(param)
    EdV = (λ*V*(log(k/V)))*step
    p = EdV/V
    for i in 1:length(agents)
        add = rand(Float64) < p
        if add
            push!(agents, :V)
        end        
    end
    return agents
end
agents = [:V]
day = 0
dt = 1/8
yet = false
for i in 1:ceil(30/dt)
    println(day, ":\t", length(agents))
    agents = transition_stoch!(agents, (λ, k), dt)
    day += dt
    if length(agents) > 800 && !false
        println("============\n$day:\t", length(agents), "\n================")
        yet = true
        break
    end
    if length(agents) >= k
        break
    end
end

0:	1
0.125:	1
0.25:	1
0.375:	2
0.5:	4
0.625:	7
0.75:	10
0.875:	13
1.0:	18
1.125:	24
1.25:	31
1.375:	39
1.5:	51
1.625:	65
1.75:	78
1.875:	90
2.0:	101
2.125:	121
2.25:	142
2.375:	175
2.5:	200
2.625:	224
2.75:	243
2.875:	269
3.0:	297
3.125:	324
3.25:	365
3.375:	397
3.5:	429
3.625:	461
3.75:	488
3.875:	522
4.0:	552
4.125:	581
4.25:	607
4.375:	628
4.5:	654
4.625:	677
4.75:	694
4.875:	725
5.0:	737
5.125:	757
5.25:	768
5.375:	793
5.5:	810


In [16]:
yet = false
for (i, u) in enumerate(sol)
    t = (tdomain[2]-tdomain[1])i/length(sol)
    if u > 800 && ! yet
        println("$t:\t", u)
        yet = true
    end
end

5.405940594059406:	802.7945435374461


In [17]:
using SemanticModels
import SemanticModels: model

┌ Info: Precompiling SemanticModels [f5ac2a72-33c7-5caf-b863-f02fefdcf428]
└ @ Base loading.jl:1260
  ** incremental compilation may be fatally broken for this module **



In [18]:
using SemanticModels.ExprModels
using SemanticModels.ExprModels.ExpODEModels
using SemanticModels.ExprModels.Transformations
import SemanticModels.ExprModels.ExpODEModels: ExpODEModel

In [20]:
code = quote module Gompertz
    using DifferentialEquations
function f(du, u,t, p)
    λ = first(p)
    k = last(p)
    return λ*u*(1-(u/k))
end
function main()
V₀ = 1.0
λ = log(2)
k = 1000
p = (λ, k)


tdomain = (0, 21)
prob = ODEProblem(f, p, V₀, tdomain)
sol = solve(prob)
end
end
end
m = model(ExpODEModel, code.args[2])

BoundsError: BoundsError: attempt to access 1-element Array{Any,1} at index [2]

In [21]:
using ModelingToolkit

ArgumentError: ArgumentError: Package ModelingToolkit not found in current path:
- Run `import Pkg; Pkg.add("ModelingToolkit")` to install the ModelingToolkit package.


In [205]:
h(x) = head(x)
h(x::Symbol) = x

@variables u λ k
Transformations.postwalk(m.funcs[1]) do x
    try
        if isa(x, Expr) && h(x) == :return
            println(x, " ",h(x))
            @show factor = x.args[end].args[end]
            
        else
            if isa(x, Symbol)
                @variables 
            end
    finally
        x
    end
    x
end

return λ * u * (1 - u / k) return
factor = (x.args[end]).args[end] = :(1 - u / k)


:(function f(du, u, t, p)
      #= In[198]:4 =#
      λ = first(p)
      #= In[198]:5 =#
      k = last(p)
      #= In[198]:6 =#
      return λ * u * (1 - u / k)
  end)

In [197]:
m.funcs[1]

:(function f(du, u, t, p)
      #= In[195]:4 =#
      λ = first(p)
      #= In[195]:5 =#
      k = last(p)
      #= In[195]:6 =#
      return λ * u * (1 - u / k)
  end)

In [210]:
@variables V λ k u t

(V(), λ(), k(), u(), t())

In [212]:
dvdt = f(V, t, (λ, k))

(λ() * V()) * log(k() / V())

In [213]:
dump(dvdt)


Operation
  op: * (function of type typeof(*))
  args: Array{Expression}((2,)) Expression[λ() * V(), log(k() / V())]


In [216]:
ModelingToolkit.vars(dvdt/V)

Set(Variable[k, V, λ])

In [218]:
dvdt/V

((λ() * V()) * log(k() / V())) / V()

In [239]:
# import Base: rand
# ModelingToolkit.@register rand(x)
# Operation(ifelse, [rand(V) < dvdt/V, 1, 0])
ModelingToolkit.@register Bernoulli(p)
Bernoulli(p) = ifelse(rand() < p, 0, 1)

Bernoulli (generic function with 2 methods)

In [240]:
Bernoulli(dvdt/V)

Bernoulli(((λ() * V()) * log(k() / V())) / V())

In [245]:
function transition_stoch!(agents, dudt, param, step)
    V = length(agents)
    λ, k = first(param), last(param)
    EdV = dudt*step
    p = EdV/V
    add = Bernoulli(p)
    @show add
    for i in 1:length(agents)
        if @show(add, (V=length(agents), λ=first(param), k=last(param)))
            push!(agents, :V)
        end
    end
    return agents
end

transition_stoch! (generic function with 3 methods)

In [246]:
@variables abmdt

(abmdt(),)

In [247]:
transition_stoch!(agents, dvdt, (λ, k), abmdt)

add = Bernoulli((((λ() * V()) * log(k() / V())) * abmdt()) / 801)
add = Bernoulli((((λ() * V()) * log(k() / V())) * abmdt()) / 801)
(V = length(agents), λ = first(param), k = last(param)) = (V = 801, λ = λ(), k = k())


TypeError: TypeError: non-boolean (NamedTuple{(:V, :λ, :k),Tuple{Int64,Operation,Operation}}) used in boolean context